# VacationPy
----

#### Author: Joey Picca

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import gkey


In [2]:
# Access maps with unique API key
gmaps.configure(api_key=gkey)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
cityWx = pd.read_csv('../output_data/wxdata.csv')

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
coords = cityWx[['lat(deg)','lon(deg)']]
humidity = cityWx["humidity(%)"].astype(int)

In [5]:
centerCoord = (33.683969, -53.389827)

# Plot Heatmap
fig = gmaps.figure(center=centerCoord,zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(coords, weights=humidity, 
                                 dissipating=False, max_intensity=110,
                                 point_radius=3)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [41]:
optimalWx = cityWx[(cityWx['temp(f)'] > 75) & (cityWx['temp(f)'] < 85) 
                    & (cityWx['humidity(%)'] > 40) & (cityWx['humidity(%)'] < 80) 
                    & (cityWx['cloudcover(%)'] < 60) & (cityWx['wind(mph)'] < 20) 
                    & (cityWx['wind(mph)'] > 10)]

optimalWx.reset_index(drop=True,inplace=True)
optimalWx

,City,Country,lat(deg),lon(deg),Date,temp(f),humidity(%),cloudcover(%),wind(mph)
0,Rikitea,PF,-23.12,-134.97,1579627940,77.59,78,26,10.87
1,Atuona,PF,-9.80,-139.03,1579627956,80.80,72,31,13.91
2,Arraial do Cabo,BR,-22.97,-42.03,1579627964,82.40,69,40,11.41
3,Sao Jose da Coroa Grande,BR,-8.90,-35.15,1579627966,81.55,78,0,13.85
4,Puerto Ayora,EC,-0.74,-90.35,1579627972,84.20,62,40,10.29
5,Cape Town,ZA,-33.93,18.42,1579627800,75.99,60,20,12.75
6,Ancud,CL,-41.87,-73.82,1579627981,75.20,50,0,14.99
7,Kavieng,PG,-2.57,150.80,1579627981,82.71,79,0,13.31
8,Vung Tau,VN,10.35,107.08,1579627995,77.61,72,50,15.88
9,Hobyo,SO,5.35,48.53,1579628032,77.20,77,40,19.93


In [25]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": gkey
}

# Use the lat/lng we recovered to identify airports
for index, row in optimalWx.iloc[0:1,:].iterrows(): #.iterrows():
    
    # get lat, lng from df
    lat = row["lat(deg)"]
    lng = row["lon(deg)"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    response = requests.get(base_url, params=params)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [54]:
hotel_df = optimalWx[['City','Country','lat(deg)','lon(deg)']]

hotel_df['Hotel Name'] = ""
hotel_df

/Users/josephpicca/anaconda/envs/grib/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,City,Country,lat(deg),lon(deg),Hotel Name
0,Rikitea,PF,-23.12,-134.97,
1,Atuona,PF,-9.80,-139.03,
2,Arraial do Cabo,BR,-22.97,-42.03,
3,Sao Jose da Coroa Grande,BR,-8.90,-35.15,
4,Puerto Ayora,EC,-0.74,-90.35,
5,Cape Town,ZA,-33.93,18.42,
6,Ancud,CL,-41.87,-73.82,
7,Kavieng,PG,-2.57,150.80,
8,Vung Tau,VN,10.35,107.08,
9,Hobyo,SO,5.35,48.53,


In [55]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": gkey
}

for index, row in hotel_df.iterrows():
    
    # get lat, lng from df
    lat = row["lat(deg)"]
    lng = row["lon(deg)"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    print(f'Querying Maps API for hotel data near {row["City"]}...')
    # make request and print url
    response = requests.get(base_url, params=params)
    
    try:
        hotel_df.loc[index,'Hotel Name'] = response.json()['results'][0]['name']
    except (IndexError):
        print(f'No results for: {row["City"]}')

Querying Maps API for hotel data near Rikitea...
Querying Maps API for hotel data near Atuona...
Querying Maps API for hotel data near Arraial do Cabo...
Querying Maps API for hotel data near Sao Jose da Coroa Grande...
Querying Maps API for hotel data near Puerto Ayora...
Querying Maps API for hotel data near Cape Town...
Querying Maps API for hotel data near Ancud...
Querying Maps API for hotel data near Kavieng...
Querying Maps API for hotel data near Vung Tau...
Querying Maps API for hotel data near Hobyo...
No results for: Hobyo
Querying Maps API for hotel data near Maragogi...
Querying Maps API for hotel data near Kokopo...
Querying Maps API for hotel data near Huarmey...
Querying Maps API for hotel data near Dhidhdhoo...
No results for: Dhidhdhoo
Querying Maps API for hotel data near Bathsheba...


In [56]:
hotel_df['Hotel Name'] = hotel_df['Hotel Name'].replace('', np.nan)
hotel_df.dropna(how='any',inplace=True)
hotel_df

/Users/josephpicca/anaconda/envs/grib/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/josephpicca/anaconda/envs/grib/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,Country,lat(deg),lon(deg),Hotel Name
0,Rikitea,PF,-23.12,-134.97,Pension Maro'i
1,Atuona,PF,-9.80,-139.03,CHALETS HANAKEE
2,Arraial do Cabo,BR,-22.97,-42.03,Hotel da Canoa
3,Sao Jose da Coroa Grande,BR,-8.90,-35.15,Crôa Mares Hotel
4,Puerto Ayora,EC,-0.74,-90.35,Hotel Solymar Galapagos
5,Cape Town,ZA,-33.93,18.42,Mojo Hotel
6,Ancud,CL,-41.87,-73.82,Hotel Y Cabañas Terrazas Vista Al Mar
7,Kavieng,PG,-2.57,150.80,Nusa Island Retreat
8,Vung Tau,VN,10.35,107.08,The IMPERIAL Hotel
10,Maragogi,BR,-9.01,-35.22,Maraga Beach Hostel


In [57]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat(deg)", "lon(deg)"]]

In [60]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))